In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import joblib

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style

# Ajuste de distribuciones
# ==============================================================================
from sklearn.neighbors import KernelDensity

# Configuración matplotlib
# ==============================================================================
plt.rcParams['figure.figsize'] = (12,8)
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

from concurrent.futures import *

from result_reader import read_results
from ponderator import ponderate_recent_events
from simulator import simulate_all

event_folder = '../classified'

In [2]:
# def single_model(event_name, sex, name, save_folder, models):
#     model = None
#     if save_folder is not None:
#         folder_path = Path(save_folder)
#         folder_path.mkdir(parents=True, exist_ok=True)
#         full_path = folder_path / f'{name}.pkl'
#         if full_path.exists():
#             model = joblib.load(full_path)
#         else:
#             model = athlete_kde_model(event_name, sex, name)
#             joblib.dump(model, full_path)
#     else:
#         model = athlete_kde_model(event_name, sex, name)
#     models.append(model)

In [3]:
data = read_results(event_folder)

In [4]:
pond_times = {
    2022: 4,
    2021: 2,
    2020: 1,
}

ponderated_data = ponderate_recent_events(data, pond_times)

In [5]:
pred = simulate_all(ponderated_data, times=1000, top=100, warnings=True)

Simulating event: 100 metros planos. Sex: male
Simulating
119
1000
Resuts:
  1: USA_fred_kerley
  2: USA_trayvon_bromell
  3: USA_marvin_bracy
  4: USA_ronnie_baker
  5: USA_michael_norman
  6: USA_micah_williams
  7: USA_christian_coleman
  8: CAN_andre_de_grasse
  9: GHA_benjamin_azamati
 10: CHN_bingtian_su
 11: ITA_lamont_marcell_jacobs
 12: JAM_oblique_seville
 13: RSA_akani_simbine
 14: KEN_ferdinand_omanyala
 15: CAN_aaron_brown
 16: USA_kenneth_bednarek
 17: USA_noah_lyles
 18: GBR_zharnel_hughes
 19: USA_elijah_hall
 20: USA_justin_gatlin
 21: JAM_yohan_blake
 22: GBR_reece_prescod
 23: USA_kyree_king
 24: NGR_favour_oghene_tejiri_ashe
 25: GHA_joseph_paul_amoah
 26: USA_isiah_young
 27: JAM_ackeem_blake
 28: CIV_ismael_kone
 29: TTO_jerod_elcock
 30: USA_kendal_williams
 31: ANT_cejhae_greene
 32: BOT_letsile_tebogo
 33: USA_brandon_carnes
 34: USA_matthew_boling
 35: USA_jo'vaughn_martin
 36: RSA_shaun_maswanganyi
 37: GBR_jeremiah_azu
 38: USA_davonte_burnett
 39: SRI_yupun

In [7]:
from prediction_saver import save_prediction

save_prediction(pred, top=3)

In [8]:
# def density_pred(kde_model: KernelDensity, samples):    
#     log_density_pred = kde_model.score_samples(X=samples.reshape(-1, 1))
#     return np.exp(log_density_pred)

    
# def plot_model(event, sex, model_name, bounds, Show=True):
#     model: KernelDensity = joblib.load(f'models/{event}/{sex}/{model_name}.pkl')
#     X = np.linspace(bounds[0], bounds[1], 200)
#     Y = density_pred(model, X)
#     plt.plot(X, Y, label=model_name)
#     if Show:
#         plt.show()

# bounds = (1700, 1900)
# event, sex = 'atl_10000m', 'female'
# plot_model(event, sex, 'ETH_letesenbet_gidey', bounds, False)
# plot_model(event, sex, 'BRN_kalkidan_gezahegne', bounds, False)
# plot_model(event, sex, 'NED_sifan_hassan', bounds, False)
# plt.legend()

# plt.show()

# model: KernelDensity = joblib.load(f'models/{event}/{sex}/NED_sifan_hassan.pkl')
# print(model.bandwidth)

In [9]:
# entry = load_entry_json()

# new_entry = {}

# for event in entry:
#     new_entry[event] = {}
#     for sex in entry[event]:
#         new_entry[event][sex] = []
        
#         for name in entry[event][sex]:
#             new_entry[event][sex].append(name.casefold())

# save_prediction(new_entry, "entry_list.json")